In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pymongo
import time


In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.FinalProject

### vehicles and years table is populated via CSV

#db.vehicles.drop()
#db.years.drop()

# Adding SUV manually to ensure script works everywhere

#b.vehicles.insert_one(
 #   {
 #       'brand': ['acura','audi','bmw','bmw','cadillac','infiniti','landrover','lexus','lexus','lincoln','mercedesbenz','porsche','volvo'],
 #       'model': ['mdx','q5','x5','x6','escalade','qx60','range-rover','rx-350','lx-570','mkx','m-class|gle-class','cayenne','xc90']
 #   }
#)




In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)
#url = 'https://www.edmunds.com/cadillac/xt5/2017/cost-to-own/'
#browser.visit(url)
#html = browser.html
#soup = BeautifulSoup(html, 'html.parser')
#soup = BeautifulSoup(html, 'lxml')


In [4]:
def scrape(url,ss):
    browser.visit(url)
    time.sleep(ss)
    html = browser.html
    #soup = BeautifulSoup(html, 'html.parser')
    soup = BeautifulSoup(html, 'lxml')
    return soup

In [5]:
def make_it_num(strInput):
    if(strInput):
        return strInput.replace("$","").replace(",","").strip()
    else:
        return 0

In [11]:
def save_car_record(detail):
    if (len(detail) > 0):
        car_year=str(detail[0])+"_final"
        car_brand=detail[1].strip()
        car_model=detail[2].strip()
        car_true_cost_own=make_it_num(detail[3].strip())
        car_cash_price=make_it_num(detail[4].strip())
        car_depreciation=[make_it_num(car_detail[i]) for i in range(6,12)]
        car_taxes=[make_it_num(car_detail[i]) for i in range(13,19)]
        car_financing=[make_it_num(car_detail[i]) for i in range(20,26)]
        car_fuel=[make_it_num(car_detail[i]) for i in range(27,33)]
        car_insurance=[make_it_num(car_detail[i]) for i in range(34,40)]
        car_maintenance=[make_it_num(car_detail[i]) for i in range(41,47)]
        car_repairs=[make_it_num(car_detail[i]) for i in range(48,54)]
        car_true_cost_5yr=[make_it_num(car_detail[i]) for i in range(55,61)]
        
        mongo_record={'year':car_year,
                       'brand':car_brand,
                       'model':car_model,
                       'owning_cost':car_true_cost_own,
                       'cash_price':car_cash_price,
                       'depreciation':car_depreciation,
                       'taxes':car_taxes,
                       'financing':car_financing,
                       'fuel':car_fuel,
                       'insurance':car_insurance,
                       'maintenance':car_maintenance,
                       'repairs':car_repairs,
                       'true_cost_5yr':car_true_cost_5yr
                     }
        #print(mongo_record)
        db[car_year].insert_one(mongo_record)
        return f"---------------------Record inserted successfully for {car_year} {car_brand} {car_model}"
        
    else:
        print("Nothing to save")
        
    

In [ ]:
###################################### MAIN PROGRAM ########################################
########################   set below three variables to scrape data ########################
#############################################################################################
#years=db.years.find(no_cursor_timeout=True)
vs =  list(db.vehicles.find())
sleep_seconds=3
##Output variable###
## This will contain complete record as list
## where 
### Index 0 - car year
### Index 1 - brand
### Index 2 - Model
### Index 3 - true cost to own
### Index 4 - total cash price
### Index5 onward - TCO details
try:
    for year in range(2015,2019):
        c_year=str(year)
        for vehicle in vs:
            car_detail=[]
            c_brand=vehicle['urlmake']
            c_model=str(vehicle['urlmodel'])
            print(f"-----------------------------------------------Checking for {c_year} {c_brand} {c_model}")

            car_detail.append(c_year)
            car_detail.append(c_brand)
            car_detail.append(c_model)

            if(c_year=='2015' and c_brand=='porsche'):
                url='https://www.edmunds.com/porsche/cayenne/2015/st-200707125/cost-to-own/'
            else:
                url=f'https://www.edmunds.com/{c_brand}/{c_model}/{c_year}/cost-to-own/'

            print(url)

            #Finding MSRP
            try:
                soup_output=scrape(url,sleep_seconds)
                msrp_results = soup_output.find("div", class_="grid-60").find_all('ul')
                for msrp_result in msrp_results:
                    #print(msrp_result.find_all('li')[1].text.replace("$","").replace(",","")  )
                    car_detail.append(msrp_result.find_all('li')[1].text)

                #finding TCO details
                tco_results = soup_output.find('ul',id="tco_detail_data").find_all('li')
                for i in range(8,len(tco_results),8):
                    records = tco_results[i].find('ul').find_all('li')

                    for record in records:
                        #print(record.text)
                        car_detail.append(record.text)
                print(save_car_record(car_detail))

            except Exception as e:
                pass
    vs.close()            
except Exception as e:
    years.close()
    vehicles.close()
    pass        
        

-----------------------------------------------Checking for 2015 chevrolet sonic
https://www.edmunds.com/chevrolet/sonic/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final chevrolet sonic
-----------------------------------------------Checking for 2015 mitsubishi montero
https://www.edmunds.com/mitsubishi/montero/2015/cost-to-own/
-----------------------------------------------Checking for 2015 scion xb
https://www.edmunds.com/scion/xb/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final scion xb
-----------------------------------------------Checking for 2015 hyundai accent
https://www.edmunds.com/hyundai/accent/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final hyundai accent
-----------------------------------------------Checking for 2015 rolls-royce phantom
https://www.edmunds.com/rolls-royce/phantom/2015/cost-to-own/
-----------------------------------------------Checking for 2015 dodge c

-----------------------------------------------Checking for 2015 chrysler pacifica-hybrid
https://www.edmunds.com/chrysler/pacifica-hybrid/2015/cost-to-own/
-----------------------------------------------Checking for 2015 dodge charger
https://www.edmunds.com/dodge/charger/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final dodge charger
-----------------------------------------------Checking for 2015 ford fiesta
https://www.edmunds.com/ford/fiesta/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final ford fiesta
-----------------------------------------------Checking for 2015 jeep grand-cherokee
https://www.edmunds.com/jeep/grand-cherokee/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final jeep grand-cherokee
-----------------------------------------------Checking for 2015 ram 3500
https://www.edmunds.com/ram/3500/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final

-----------------------------------------------Checking for 2015 chrysler pacifica
https://www.edmunds.com/chrysler/pacifica/2015/cost-to-own/
-----------------------------------------------Checking for 2015 dodge grand-caravan
https://www.edmunds.com/dodge/grand-caravan/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final dodge grand-caravan
-----------------------------------------------Checking for 2015 ford five-hundred
https://www.edmunds.com/ford/five-hundred/2015/cost-to-own/
-----------------------------------------------Checking for 2015 volvo s60
https://www.edmunds.com/volvo/s60/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final volvo s60
-----------------------------------------------Checking for 2015 chevrolet cobalt
https://www.edmunds.com/chevrolet/cobalt/2015/cost-to-own/
-----------------------------------------------Checking for 2015 volkswagen tiguan
https://www.edmunds.com/volkswagen/tiguan/2015/cost-to-o

---------------------Record inserted successfully for 2015_final ford mustang
-----------------------------------------------Checking for 2015 mercedesbenz slk-class
https://www.edmunds.com/mercedesbenz/slk-class/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final mercedesbenz slk-class
-----------------------------------------------Checking for 2015 fiat 500
https://www.edmunds.com/fiat/500/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final fiat 500
-----------------------------------------------Checking for 2015 ford fusion
https://www.edmunds.com/ford/fusion/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final ford fusion
-----------------------------------------------Checking for 2015 nissan armada
https://www.edmunds.com/nissan/armada/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final nissan armada
-----------------------------------------------Checking for 

-----------------------------------------------Checking for 2015 jeep renegade
https://www.edmunds.com/jeep/renegade/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final jeep renegade
-----------------------------------------------Checking for 2015 toyota avalon
https://www.edmunds.com/toyota/avalon/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final toyota avalon
-----------------------------------------------Checking for 2015 rolls-royce dawn
https://www.edmunds.com/rolls-royce/dawn/2015/cost-to-own/
-----------------------------------------------Checking for 2015 rolls-royce ghost
https://www.edmunds.com/rolls-royce/ghost/2015/cost-to-own/
-----------------------------------------------Checking for 2015 rolls-royce ghost-series-ii
https://www.edmunds.com/rolls-royce/ghost-series-ii/2015/cost-to-own/
-----------------------------------------------Checking for 2015 rolls-royce wraith
https://www.edmunds.com/rolls-royce/wrait

---------------------Record inserted successfully for 2015_final dodge durango
-----------------------------------------------Checking for 2015 ford ecosport
https://www.edmunds.com/ford/ecosport/2015/cost-to-own/
-----------------------------------------------Checking for 2015 ford taurus
https://www.edmunds.com/ford/taurus/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final ford taurus
-----------------------------------------------Checking for 2015 ford c-max-hybrid
https://www.edmunds.com/ford/c-max-hybrid/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final ford c-max-hybrid
-----------------------------------------------Checking for 2015 ford transit-connect
https://www.edmunds.com/ford/transit-connect/2015/cost-to-own/
---------------------Record inserted successfully for 2015_final ford transit-connect
-----------------------------------------------Checking for 2015 ford focus-st
https://www.edmunds.com/ford/focus-st/

In [ ]:
db.rrecord.renameCollection("record")

mongoexport --db FinalProject --collection 2015 --out 2015.json
